# Import Libraries

In [ ]:
import pandas as pd
import tagui as t
import time

# Knowledge Base Setup

In [ ]:
KB = 'Knowledge Base_Complete.xlsx'
df_time = pd.read_excel(KB,"Travel Time",index_col = 0)
df_ave_rent = pd.read_excel(KB,"Ave Rental",index_col = 0)

In [ ]:
df_time.head()

In [ ]:
df_ave_rent = df_ave_rent.drop('MRT Code',axis=1)
df_ave_rent.head()

In [ ]:
df_ave_rent_minmax = df_ave_rent.drop('MRT Unique',axis=1)
df_ave_rent_ori_processed = df_ave_rent_minmax[df_ave_rent_minmax['Average Room Rental Prices (S$)'] != "No data"]
df_ave_rent_minmax = df_ave_rent_minmax[df_ave_rent_minmax['Average Room Rental Prices (S$)'] != "No data"]

for column in df_ave_rent_minmax.columns:
    if column == "Station Name":
        continue

    else:
        df_ave_rent_minmax[column] = (df_ave_rent_minmax[column] - df_ave_rent_minmax[column].min())/ (df_ave_rent_minmax[column].max() - df_ave_rent_minmax[column].min())

df_ave_rent_minmax.head()


In [ ]:
# Max and Mins for normalization purposes
time_min = 0
time_max = 97

room_rent_min = df_ave_rent_ori_processed['Average Room Rental Prices (S$)'].min()
room_rent_max = df_ave_rent_ori_processed['Average Room Rental Prices (S$)'].max()
house_rent_min = df_ave_rent_ori_processed['Average Overall Whole-house Rental'].min()
house_rent_max = df_ave_rent_ori_processed['Average Overall Whole-house Rental'].max()

In [ ]:
df_index = df_ave_rent.drop(["MRT Unique",
                             "Average Room Rental Prices (S$)",
                             "Average 3-Room House Rental Prices (S$)",
                             "Average 4-Room House Rental Prices (S$)",
                             "Average 5-Room House Rental Prices (S$)",
                            "Average Overall Whole-house Rental"],axis=1)
df_index.head()

In [ ]:
df_MRT_code = pd.read_excel('Knowledge Base_Complete.xlsx',"Ave Rental",index_col = 0)
df_MRT_code = df_MRT_code.drop(['MRT Unique',
                                "Average Room Rental Prices (S$)",
                             "Average 3-Room House Rental Prices (S$)",
                             "Average 4-Room House Rental Prices (S$)",
                             "Average 5-Room House Rental Prices (S$)",
                            "Average Overall Whole-house Rental"],axis=1)

df_MRT_code = df_MRT_code.rename(columns={'MRT Code':'MRT ID'})
df_MRT_code = df_MRT_code.set_index('Station Name')
df_MRT_code.head(50)
# df_MRT_code = df_ave_rent.drop(["Average Room Rental Prices (S$)",
#                              "Average 3-Room House Rental Prices (S$)",
#                              "Average 4-Room House Rental Prices (S$)",
#                              "Average 5-Room House Rental Prices (S$)",
#                             "Average Overall Whole-house Rental"],axis=1)

# df_MRT_code = df_MRT_code.rename(columns={'MRT Unique':'MRT ID'})
# df_MRT_code = df_MRT_code.set_index('Station Name')
# df_MRT_code.head()

# Dictionary and Functions Setup

In [ ]:
# MRT Index by index (e.g. 1: Admiralty, 2: Aljunied,  etc)
MRT_index_name = df_index.to_dict()

# MRT index by name (e.g. Admiralty: 1, Aljunied: 2, etc)
MRT_name_index = {value: key for key, value in MRT_index_name['Station Name'].items()}

print(MRT_index_name)
print()

In [ ]:
MRT_codes = df_MRT_code.to_dict()
print(MRT_codes)

# Utilities

In [ ]:
def scoring(price,
            travel_time,
            budget,
            house_type,
            comfortable_travel_time):
    
    housing_dict = {'room':{'min':917,
                           'max':3200},             
            'entire house':{'min':2317,
                            'max':3800}}
    
    hd = housing_dict[house_type.lower()]
    norm_price = (price - hd['min']/(hd['max']-hd['min']))
    norm_travel_time = travel_time/97 # time_max = 97, time_min = 0
    
    # Budget Bonus scoring to boost properties around the budget set.
    # Prevents the situation where lowest price is always favoured regardless of budget.
    if price>0.7*budget and price< 1.1*budget:
        budget_bonus = 1.5
    elif price>0.5*budget and price<0.7*budget:
        budget_bonus = 1
    elif price>1.5*budget and price<2.0*budget:
        budget_bonus = -1
    elif price>2.0*budget:
        budget_bonus = -2.1
    elif price>2.5*budget:
        budget_bonus = -5
    else:
        budget_bonus = 0
    
    # Travel Time Penalty if exceed the 
    if float(travel_time)>comfortable_travel_time:
        time_penalty = 0.5*float(travel_time-comfortable_travel_time)
    else:
        time_penalty = 0
    
    # in the nearest MRT formula, the bigger weight is on travel time as the price reflects an average
    
    score = (1/(0.65*norm_price+0.2*norm_travel_time+1))*10 + budget_bonus - time_penalty
    
    return score

In [ ]:
def lets_wait(target_element):
    start_time = time.time()
    time_lapsed = 0
    while not t.present(target_element) and time_lapsed < 8:
        t.wait(1)
        end_time = time.time()
        time_lapsed = end_time - start_time
    return target_element

In [ ]:
def time_converter(input_data):
    input_time = [time for time in input_data.split() if time.isnumeric()]
    if input_time ==[]:
        return 10000
    else:
        input_unit = input_data.split()
        if len(input_time) == 2:
            hour_to_min = int(input_time[0]) * 60
            total_time = hour_to_min + int(input_time[1])

        elif len(input_time) == 1 and ("hr" not in input_unit or 'h' not in input_unit):
            total_time = (input_time[0])

        elif len(input_time) == 1:
            total_time = int(input_time[0]) * 60

        else:
            total_time = 0
    
    return total_time

In [ ]:
def shortlist_generation(prop_titles,prop_addresses,prop_prices,travel_times,links,scores):
    if (len(prop_titles)+len(prop_addresses)+len(prop_prices)+len(travel_times)+len(links))/5 != len(scores):
        return "Scrape does not tally"
#         return len(prop_titles),len(prop_addresses),len(prop_prices),len(travel_times),len(links), len(scores)
    else:
        shortlist = pd.DataFrame({'Property' : prop_titles,
                                          'Address' : prop_addresses,
                                          'Price (S$/mo)' : prop_prices,
                                          'Travel Time (mins)': travel_times,
                                         'Property Score': scores,
                                         'Links' : links})

        shortlist = shortlist.sort_values(by=['Property Score'],ascending=False)

        return shortlist

In [ ]:
def ave_time_generation(address,CoA,CoA_Name,timecheck_index):
    tt_total = 0.0
    for i in range(len(list(CoA))):
        tt = time_check(address,f"Singapore {CoA[i]}",CoA_Name[i],timecheck_index)
        tt_total += float(tt)
    ave_time = float(tt_total) / len(CoA)
    return ave_time

In [ ]:
def dummy_fillers(pipeline, shortlist_num = 5):
    chromosome_1 = [i+1 for i in range(3)] # MRT Search Method
    chromosome_2 = [i+1 for i in range(8)] # Sort Method
    chromosome_3 = [i+1 for i in range(2)] # Scraping Method
    chromosome_4 = [i+1 for i in range(3)] # Timecheck Method
    
    all_chromos = [chromosome_1, chromosome_2, chromosome_3, chromosome_4]
    
    for pipes, chromos in zip(pipeline, all_chromos):
        if pipes not in chromos:
            dummy_list = []
            dummy_list = dummy_list + [10000]*(shortlist_num)
            prop_titles,prop_addresses,prop_prices,travel_times,links = [dummy_list.copy() for _ in range(5)]
            scores = [] + [0]*(shortlist_num)
            shortlist = shortlist_generation(prop_titles,prop_addresses,prop_prices,travel_times,links,scores)
            time_lapsed = 1000
            break

        else:
            shortlist, time_lapsed = "Pass","Pass"
            
    return shortlist, time_lapsed

# Scraping and Shortlisting Codes

In [ ]:
def URL_generator(max_price, house_type, mrt_name, MRT_method_index, sort_method_index, website='property guru'):
    
    global df_MRT_code
    url_adapted = mrt_name.replace(" ","+")
    MRT_code = df_MRT_code['MRT ID'][mrt_name]
    split_code = MRT_code.split()

    url_mrt = ""
    concat_code = ""
    for code in split_code:
        url_mrt += f"&MRT_STATIONS[]={code}"
        concat_code += f"/{code}"
    concat_code = concat_code[1:]
    url_mrt = url_mrt[1:]
    
    url_details = {'property guru':{'url':'https://www.propertyguru.com.sg/',
                               'rental housing':'property-for-rent?market=residential&listing_type=rent&',
                               'max price':f'maxprice={str(max_price)}&',
                                   'room':'beds[]=-1&',
                                   'entire house':'beds[]=0&beds[]=1&beds[]=2&beds[]=3&beds[]=4&beds[]=5&',
                                   'search true':'search=true',
                                   'MRT search method':{'1':f"{url_mrt}&freetext={concat_code}+{url_adapted}+MRT&",
                                                       '2':f"freetext={url_adapted}+MRT&",
                                                       '3':f"freetext={url_adapted}&"},
                                   'sort method':{"1": '', # Recommended
                                                  "2": '&sort=date&order=desc', # Newest
                                                  "3": '&sort=price&order=asc', # Lowest Price
                                                  "4": '&sort=price&order=desc', # Highest Price
                                                  "5": '&sort=psf&order=asc', # PSF (low-high)
                                                  "6": '&sort=psf&order=desc', # PSF (high-low)
                                                  "7": '&sort=size&order=asc', # Size (low-high)
                                                  "8": '&sort=size&order=desc'}}} # Size (high-low)
    
    
    ud = url_details[website.lower()]
    
    base = ud['url']
    rental_housing = ud['rental housing'] 
    max_price = ud['max price']
    house_type = ud[house_type.lower()]
    mrt_search = ud['MRT search method'][str(MRT_method_index)]
    ending = ud['search true']
    sort_method = ud['sort method'][str(sort_method_index)]
    
    full_url = "".join([base,rental_housing,max_price,house_type,mrt_search,ending,sort_method])
    return full_url

In [ ]:
def time_check(start_point,end_point,end_point_name,option_index):
    option_index = str(option_index)
    
    gothere_startpoint = start_point.replace(" ","%20")
    gothere_endpoint = end_point.replace(" ","%20")
    
    site_details = {"1":{"url":'https://www.google.com/maps/dir///',
                        "start box":'(//input[@class="tactile-searchbox-input"])[1]',
                        "end box":'(//input[@class="tactile-searchbox-input"])[2]',
                        "search button":'(//button[@data-tooltip="Search"])[2]',
                        "public transport option": '//img[@data-tooltip="Transit"]',
                        "depart by options":'//div[@id=":3"]',
                        "select depart at":'//div[@id=":1"]',
                        "departing time box":'//input[@name="transit-time"]',
                        "12:30pm option":'//div[@id=":p"]',
                        "travel times":'//div[@class="Fk3sm fontHeadlineSmall"]'},
                    
                   "2":{"url":'https://www.streetdirectory.com/travel/',
                        "close ad":'//a[@class="btn_close"]',
                        "start box":'//input[@id="travel_from"]',
                        "end box":'//input[@id="travel_to"]',
                        "search button":'//a[@id="go_button"]',
                        "confirm locations":'(//tr[@class="information"])[1]',
                        "select depart at":'//input[@id="time"]',
                        "departing am/pm":'(//a[@class="ptTimeSelectHr ui-state-default"])[2]',
                        "12:30pm option":'(//a[@class="ptTimeSelectHr ui-state-default"])[14]',
                        "confirm timing":'//a[@onclick="jQuery.ptTimeSelect.setTime()"]'},
                   
                    "3": {"url":f'https://gothere.sg/maps#q:from%20{gothere_startpoint}%20to%20{gothere_endpoint}%20at%2012%3A30pm',
                        "search box":'//input[@id="q"]',
                         "search button": '//input[@id="ss"]'}}
    
    min_time = 1000
    # method begins here:
    tc = site_details[option_index]
    t.url(tc['url'])
    
    if option_index == "1": # Google Maps
        t.type(tc["start box"],start_point)
        t.type(tc["end box"], end_point)
        t.click(tc["search button"])
        
        t.click(lets_wait(tc["public transport option"]))
        t.wait(3)
        t.click(tc["depart by options"])
        t.click(lets_wait(tc["select depart at"]))
        t.type(tc["departing time box"],"[clear]")
        t.type(tc["departing time box"],"[clear]12:30pm[enter]")
        t.wait(3)
        
        for i in range(1,t.count(tc["travel times"])+1):
            travel_time = time_converter(t.read(f'({tc["travel times"]})[{i}]'))
            if int(travel_time) < int(min_time):
                min_time = travel_time
    
    elif option_index == "2": # Street Directory
        start_time = time.time()
        time_lapse = 0
        while not t.present(tc["close ad"]) and time_lapse < 8 :
            t.wait(1)
            end_time = time.time()
            time_lapse = end_time - start_time
        
        if t.present(tc["close ad"]):
            t.click(tc["close ad"])
            
            
        t.type(tc["start box"],start_point+'[enter]')
        t.type(tc["end box"],end_point_name+'[enter]')
        t.click(tc["search button"])
        t.wait(2)

        while t.present(tc["confirm locations"]):
            t.click(tc["confirm locations"])
            t.wait(2)
        
        for process in ["select depart at", "departing am/pm", "12:30pm option", "confirm timing"]:
            t.click(tc[process])

        for mode in ['bus','busmrt']:
            travel_time = time_converter(t.read(f'//span[@id="{mode}_time"]'))
            if int(travel_time) < int(min_time):
                min_time = travel_time
    
    
    elif option_index == "3": # gothere.sg
        t.wait(2)
        t.click(tc['search button'])

        for mode in ["train","bus"]:
            t.click(f'//li[@id="{mode}"]')
            t.wait(1)
            
            for i in range(1,t.count(f'//div[@mode="{mode}"]/p[@class="n1"]')+1):
                travel_time = time_converter(t.read(f'(//div[@mode="{mode}"]/p[@class="n1"])[{i}]'))
                if int(travel_time) < int(min_time):
                    min_time = travel_time
        
    return min_time
    

In [ ]:
def shortlist_properties(url, CoA, CoA_Name, budget, house_type, comfortable_travel_time, 
                          search_method_index, timecheck_index, time_average, 
                         shortlist_num = 5, website='property guru', no_change_max = 7):
    '''Function to scrape website and shortlist properties based on travel time and price. 
    CoA and CoA_name here should be in a list format'''
    
    website_details = {'property guru':{'no results':'''//h3[text()="Sorry, we couldn't find any results for:"])''',
                                        'search box':'//input[@placeholder="Search Location"]',
                                        'prop title':'//div[@class="gallery-container"]/a/@title',
                                       'prop address':'//span[@itemprop="streetAddress"]',
                                        'prop price':'//li[@class="list-price pull-left"]/span[@class="price"]',
                                        'prop link':'(//div[@class="gallery-container"]/a/@href)',
                                        'next page':'//li[@class="pagination-next"]/a',
                                       'stop code': '//span[@data-dobid="hdw"]'}}
    
    wd = website_details[website]
    count = 1
    t.wait(2)
    t.init()
    t.url(url)
    
    # this loop is to prevent captcha from stopping the process. User needs to click the verify in order to proceed.
    while not t.present(wd['search box']):
        t.wait(1)
    
    
    next_page = True
    t.wait(4)
    if t.present(wd['no results']):
        return "Sorry, we couldn't find any results. Please update your search criteria"
    
    elif str(search_method_index) == "1":
        # Initialisation
        prop_titles, prop_addresses, prop_prices, links, travel_times, scores, temp_address_memory, temp_time_memory = [[] for _ in range(8)]

        while next_page:
            lets_wait(wd['prop address'])
            prop_no = t.count(wd['prop address'])
            for i in range(1, prop_no + 1):
                # Reading
                prop_title = t.read(f"({wd['prop title']})[{i}]")
                prop_address = t.read(f"({wd['prop address']})[{i}]")
                price = t.read(f"({wd['prop price']})[{i}]")
                price = int(price.replace(',',''))
                link = t.read(f"{wd['prop link']}[{i}]")

                # Appending
                prop_titles.append(prop_title)
                prop_addresses.append(prop_address)
                prop_prices.append(price)
                links.append(link)
                

            if t.present(wd['next page']):
                count += 1
                t.url(f"{url[:49]}/{count}{url[49:]}")                    
#                     t.click(wd['next page'])
                t.wait(3)

            else:
                next_page = False

        
        for address in prop_addresses:
            if address not in temp_address_memory:
                ave_time = ave_time_generation(address,CoA,CoA_Name,timecheck_index)
                travel_times.append(ave_time)
                temp_address_memory.append(address) # Memory component to save redundant searches
                temp_time_memory.append(ave_time) # Memory to recall the saved address
            else:
                travel_times.append(temp_time_memory[temp_address_memory.index(address)])

        for i in range(len(prop_addresses)):
            prop_score = scoring(prop_prices[i],travel_times[i],budget,house_type,comfortable_travel_time)
            scores.append(prop_score)

        
        shortlist = shortlist_generation(prop_titles,prop_addresses,prop_prices,travel_times,links,scores)
        shortlist = shortlist[:shortlist_num]
        
    elif str(search_method_index) == "2":
        # Initialisation
        scores = []
        scores = scores + [-1000]*(shortlist_num - len(scores))
#         print("Initial scores:",scores)
        prop_titles, prop_addresses, prop_prices, links, travel_times = [scores.copy() for _ in range(5)]
        temp_address_memory, temp_time_memory = [],[]
        no_change_count = 0

        while next_page:
            lets_wait(wd['prop address'])
            prop_no = t.count(wd['prop address'])

            for i in range(1, prop_no + 1):
                skip_timecheck = False
                prop_address = t.read(f"({wd['prop address']})[{i}]")
                price = t.read(f"({wd['prop price']})[{i}]")
                price = int(price.replace(',',''))
                prop_score = scoring(price,time_average,budget,house_type,comfortable_travel_time) #ave_travel_time here is station[1] in nearest station


                for j in range(len(scores)):
                    if prop_score > scores[j]:
                        link = t.read(f"{wd['prop link']}[{i}]")
                        prop_title = t.read(f"({wd['prop title']})[{i}]")
#                         print(prop_title)

                        # Removing initialized 0s
#                         print("Pre-removal scores:",scores)
                        scores.remove(scores[j])
                        prop_titles.remove(prop_titles[j])
                        prop_addresses.remove(prop_addresses[j])
                        prop_prices.remove(prop_prices[j])
                        links.remove(links[j])
                        travel_times.remove(travel_times[j])

                        # Appending new information
#                         scores.append(prop_score)
                        prop_titles.append(prop_title)
                        prop_addresses.append(prop_address) 
                        prop_prices.append(price)
                        links.append(link)
                        skip_timecheck = False
#                         print("Predicted Scores:",prop_score)
                        break
                        
                    else:
                        skip_timecheck = True


                if skip_timecheck:
#                     print("Current no change count before +1: ",no_change_count)
                    no_change_count += 1
                    if no_change_count > no_change_max:
                        break
                    else:
                        continue

                else:           
#                     print("Resetting no change count")
                    no_change_count = 0
#                     print(no_change_count)
                    if prop_address not in temp_address_memory:
                        current_url = t.url()
                        ave_time = ave_time_generation(prop_address,CoA,CoA_Name,timecheck_index)
                        travel_times.append(ave_time)
                        temp_address_memory.append(prop_address) # Memory component to save redundant searches
                        temp_time_memory.append(ave_time) # Memory to recall the saved address
                        t.url(current_url)
                    else:
                        travel_times.append(temp_time_memory[temp_address_memory.index(prop_address)])

                    prop_score = scoring(price,ave_time,budget,house_type,comfortable_travel_time)
                    scores.append(prop_score)
#                     print("Actual Score:",prop_score)
#                     print("Current Set of Scores:",scores)
#                     print()



            if no_change_count > no_change_max:
                shortlist = shortlist_generation(prop_titles,prop_addresses,prop_prices,travel_times,links,scores)
                break

            elif t.present(wd['next page']):
                count += 1
                t.url(f"{url[:49]}/{count}{url[49:]}")                
#                 t.click(wd['next page'])
                t.wait(3)
            else:
                shortlist = shortlist_generation(prop_titles,prop_addresses,prop_prices,travel_times,links,scores)
                next_page = False

#             shortlist = shortlist_generation(prop_titles,prop_addresses,prop_prices,travel_times,links,scores)
    else:
        shortlist = "Search Error"
    
    t.close()
    return shortlist

# Pipeline Assembly

In [ ]:
def process_pipeline(CoA,CoA_Name,budget,house_type,comfortable_travel_time, # Obtained from user 
                     time_average, mrt_name, # Obtained from for-loop calculation 
                     pipeline, no_change_max = 7,shortlist_num = 5, website='property guru'): # Set by us
    '''Function to arrange the process flow. Please note that CoA, CoA_Name, and Pipeline parameters must be in lists
    e.g of CoA = [111234,234000]
    e.g. of CoA_Name = ["Yishun Green","Lot One"]
    e.g. of Pipeline = [1,2,1,1]'''
    
    # pipeline[0] = search_method_constrint (3 methods)
    # pipeline[1] = sort_method_constraint (8 methods)
    # pipeline[2] = scrape_method_constraint (2 methods)
    # pipeline[3] = time_method_constraint (3 methods)
    
    dummy_check1, dummy_check2 = dummy_fillers(pipeline)
    if dummy_check2 == "Pass":
        start_time = time.time()

        URL = URL_generator(budget, house_type, mrt_name, pipeline[0], pipeline[1]) #
        shortlist = shortlist_properties(URL, CoA, CoA_Name, budget, house_type, comfortable_travel_time, pipeline[2], pipeline[3], time_average, no_change_max=no_change_max,shortlist_num = shortlist_num, website=website)

        end_time = time.time()
        time_lapsed = end_time - start_time
    
    else:
        shortlist, time_lapsed = dummy_check1, dummy_check2
#     print(f"Process took {time_lapsed} seconds")
    
    return shortlist, time_lapsed
    


# Beta Testing

In [ ]:
# Fixing the Parameters as Constants
CoA = ["039594"]
CoA_Name = ["Marina Square"]
budget = 1500
house_type = "Room"
comfortable_travel_time = 47
time_average = 23
mrt_name = "Potong Pasir"

In [ ]:
# pipelines = [[1,1,2,3]]

# for pipeline_solution in pipelines:
#     shortlists, time_lapsed = process_pipeline(CoA,CoA_Name,budget,
#                                             house_type,comfortable_travel_time, 
#                                             time_average, mrt_name, pipeline_solution, no_change_max=7,shortlist_num=5)

#     fitness_score = shortlists['Property Score'].sum()/time_lapsed
#     print(shortlists)
#     print(time_lapsed)
#     print(fitness_score)

# Genetic Algorithm

In [ ]:
# Step 2: Define fitness function
import random
import math

def fitness(CoA,CoA_Name,budget,house_type,comfortable_travel_time,time_average, mrt_name, pipeline_solution):

    shortlists, time_lapsed = process_pipeline(CoA,CoA_Name,budget,
                                                   house_type,comfortable_travel_time, 
                                                   time_average, mrt_name, pipeline_solution)

    print(shortlists)
    
    shortlist_scores = shortlists['Property Score'].sum()
   
    
    fitness_score = shortlist_scores /(time_lapsed)
    print(f'Fitness score for {pipeline_solution} is {fitness_score}')
    return fitness_score

In [ ]:
# Set Method constraints:
method_constraints_dict = {"0":3, # search_method_constraint
                          "1":8, # sort_method_constraint
                          "2":2, # scrape_method_constraint
                          "3":3} # time_method_constraint

# Start of Genetic Algorithm:
population_size = 10
generations = 5
mutation_rate = 0.5
crossover_probability = 0.5

# Step 1: Initialize population
population = []
for i in range(population_size):
    pipeline_solution = [random.randint(1, 3), random.randint(1, 8), random.randint(1, 2), random.randint(1, 3)]
    population.append(pipeline_solution)



temp_pipeline_memory, temp_pipeline_score_memory = [],[]    
# Step 3 to 7: Selection, crossover, mutation, and evaluation
for generation in range(generations):
    print(f' Now Generation {generation}')
    print (population)
    # Step 3: Selection and Evaluation
    selected = []
    selected_scores = []
    for i in range(population_size // 2):
        solution1 = random.choice(population)
        solution2 = random.choice(population)
        
        print(f'1st scrapping {solution1}')
        if solution1 not in temp_pipeline_memory:
            fitness1 = fitness(CoA,CoA_Name,budget,house_type,comfortable_travel_time,time_average, mrt_name,solution1)
            temp_pipeline_memory.append(solution1)
            temp_pipeline_score_memory.append(fitness1) # Memory component to save redundant searches
        else:
            fitness1 = temp_pipeline_score_memory[temp_pipeline_memory.index(solution1)]
       
        print(f'2nd scrapping {solution2}')
        if solution2 not in temp_pipeline_memory:
            fitness2 = fitness(CoA,CoA_Name,budget,house_type,comfortable_travel_time,time_average, mrt_name,solution2)
            temp_pipeline_memory.append(solution2)
            temp_pipeline_score_memory.append(fitness2) # Memory component to save redundant searches
        else:
            fitness2 = temp_pipeline_score_memory[temp_pipeline_memory.index(solution2)]
        
        if fitness1 > fitness2:
            selected.append(solution1)
            selected_scores.append(fitness1)
        else:
            selected.append(solution2)
            selected_scores.append(fitness2)
    print(selected)
    gen_score_df = pd.DataFrame({'Pipeline' : selected,
                                 'Pipeline_Scroe' : selected_scores},
                                columns=['Pipeline', 'Pipeline_Scroe'])

    gen_score_df.to_csv(f"Generations/Gen_{generation}.csv")

#### Remove Crossover to reduce complexity and 
#     Step 4: Crossover
    new_population = []
    for i in range(population_size // 2):
        solution1 = random.choice(selected)
        solution2 = random.choice(selected)
        new_solution = []
        for j in range(len(solution1)):
            if random.random() < crossover_probability:
                new_solution.append(solution1[j])
            else:
                new_solution.append(solution2[j])
        new_population.append(new_solution)

    # Step 5: Mutation
    for i in range(population_size // 2):
        for j in range(len(new_population[i])):
            if random.random() < mutation_rate:
                new_population[i][j] = random.randint(1, method_constraints_dict[str(j)])
    
    # Step 6: Evaluation
    print(f"new pop {new_population}")
    for i in range(len(new_population)):
        print (f'Now using {new_population[i]}')
        
        if new_population[i] not in temp_pipeline_memory:
            new_population_score = fitness(CoA,CoA_Name,budget,house_type,comfortable_travel_time,time_average, mrt_name,new_population[i])
            temp_pipeline_memory.append(new_population[i])
            temp_pipeline_score_memory.append(new_population_score) # Memory component to save redundant searches
        else:
            new_population_score = temp_pipeline_score_memory[temp_pipeline_memory.index(new_population[i])]
        
        selected.append(new_population[i])
        selected_scores.append(new_population_score)
        
    selected_scores, selected = zip(*sorted(zip(selected_scores, selected),reverse = True))
    population = list(selected[:math.ceil((population_size//2)*(generations-generation)/generations)])

    
# Step 8: Return best solution
best_solution = max(selected_scores)
print(best_solution)

last_gen_score_df = pd.DataFrame({'Pipeline' : selected,
                                 'Pipeline_Scroe' : selected_scores},
                                columns=['Pipeline', 'Pipeline_Scroe'])

last_gen_score_df.to_csv(f"Generations/Last_Gen.csv")